In [265]:
from environment import Environment
import numpy as np
from game import SumUnanimityGames, GlobalFeatureImportance, LocalFeatureImportance, UnsupervisedFeatureImportance
from plot import plot

In [266]:
from algorithms.CMCS import *
from algorithms.GapE import GapE
from algorithms.SAR import SAR
from algorithms.ApproShapley import ApproShapley
from algorithms.BUS import BUS
from algorithms.HybridApproBUS import SmartHybridApproBUS
from algorithms.SVARM import SVARM, StratSVARM
from algorithms.ShapleySort import ShapleySort
from algorithms.KernelSHAP import KernelSHAP
from algorithms.shap_k import SHAP_K
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [267]:
n=15
k=5
rounds=20
metric = "ratio"
env = Environment(n=n, budget=-1, metric=metric)
results = []

In [268]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [269]:
filepath_global = "datasets/Global feature importance/Bike sharing regression random forest.csv"
filepath_unsupervised = "datasets/Unsupervised Feature Importance/vf_BigFive.csv"
directory_local = "datasets/nlp_sentiment"
game = GlobalFeatureImportance(filepath=filepath_global, num_players=n, use_cached=True)
# game = LocalFeatureImportance(directory=directory_local, num_players=n)
# game = UnsupervisedFeatureImportance(filepath=filepath_unsupervised, num_players=n)
# game = SumUnanimityGames()

[0.000 33408.278 33408.278 ... 42740.689 42740.689 42734.392]
[6690.038 6690.038 2158.998 3901.994 2393.990 58.970 475.725 674.210
 1000.652 2213.801 1700.790 -253.814 -1068.698 5445.858 10651.839] 42734.391623665346


In [270]:
t_min = 30
delta = 0.01
epsilon = 0.005

In [271]:
results += [(f"SHAP@K(t_min={t_min}, delta={delta}, epsilon={epsilon})", env.evaluate_PAC(game, SHAP_K(t_min, delta, epsilon), k, rounds))]

6139
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4956
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4651
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3550
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3542
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
6547
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3771
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
2860
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
8317
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
7727
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4865
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
8928
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4494
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4062
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3800
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13

In [272]:
results += [(f"Greedy CMCS(t_min={t_min}, delta={delta}, epsilon={epsilon})", env.evaluate_PAC(game, Greedy_CMCS(t_min, delta, epsilon), k, rounds))]


9879
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
7942
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
8312
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
9196
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
9999
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
7421
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
7077
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
11356
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
9595
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
10502
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
7764
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
8520
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
9956
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
8230
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
9860
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 

In [273]:
results += [(f"CMCS@K(t_min={t_min}, delta={delta}, epsilon={epsilon})", env.evaluate_PAC(game, CMCS_at_K(t_min, delta, epsilon), k, rounds))]

3789
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3434
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4678
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3596
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4704
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4273
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
2651
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4347
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
4990
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3608
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
2976
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
5058
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3614
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3636
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
3990
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13

In [274]:
results += [(f"CMCS(t_min={t_min}, delta={delta}, epsilon={epsilon})", env.evaluate_PAC(game, CMCS(t_min, delta, epsilon), k, rounds))]

11103
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
12948
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
8875
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
10589
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
13073
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
10956
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
16014
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
13380
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
12426
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
12847
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
10975
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
12105
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
18560
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
11828
topk_real:    [ 0  1  3 13 14]
topk_approx:  [ 0  1  3 13 14]
15488
topk_real:    [ 0  1  3 13 14]
topk_approx:

In [275]:
results

[('SHAP@K(t_min=30, delta=0.01, epsilon=0.005)', (4970.6, 396.49620294193033)),
 ('Greedy CMCS(t_min=30, delta=0.01, epsilon=0.005)',
  (8944.35, 267.6146018654045)),
 ('CMCS@K(t_min=30, delta=0.01, epsilon=0.005)',
  (3876.95, 162.93643376288932)),
 ('CMCS(t_min=30, delta=0.01, epsilon=0.005)', (12844.9, 557.7142585684235))]